In [8]:
from collections import deque
from enum import Enum
import heapq

maxprio = 4
current_running_process = None

# final stats
last_process_finish_time = float('-inf')
cpu_time = 0
io_time = 0

class process_state(Enum):
    CREATED = 1
    READY = 2
    RUNNING = 3
    BLOCKED = 4
    
class process_transition(Enum):
    CREATED_TO_READY = 1
    READY_TO_RUNNING = 2
    RUNNING_TO_BLOCKED = 3
    BLOCKED_TO_READY = 4
    RUNNING_TO_READY = 5
    DONE = 6

class Process:
    def __init__(self, id, at, tc, cb, io):
        self.id = id
        
        self.at = at
        self.tc = tc
        self.cb = cb
        self.io = io
        
        self.static_priority = rand_generator.next(maxprio)
        self.dynamic_priority = self.static_priority - 1
        self.remaining_time = tc
        self.finish_time = None
        self.turnaround_time = None
        self.io_time = 0
        self.cw = 0
        
def get_random_numbers(filename:str) -> list[int]:
    random_numbers = []
    with open(filename) as f:
        random_numbers = f.readlines()
        
    randos = [int(i) for i in random_numbers[1:]]
    return randos

def get_process_array(filename:str) -> deque[Process]:
    process_array = []
    with open(filename) as f:
        pid = 0
        for line in f:
            at, tc, cb, io = [int(i) for i in line.split()]
            process_array.append(Process(pid, at, tc, cb, io))
            pid+=1
    return process_array

def print_summary(scheduler, process_arr):
    global last_process_finish_time, cpu_time, io_time
    total_cpu_time = sum(p.tc - p.remaining_time for p in process_arr if p.finish_time is not None)
    total_elapsed_time = last_process_finish_time - process_arr[0].at
    cpu_util = total_cpu_time / total_elapsed_time * 100

    io_util = io_time / last_process_finish_time * 100.
    avg_tat = sum([p.turnaround_time for p in process_arr]) / len(process_arr)
    avg_cw = sum([p.cw for p in process_arr]) / len(process_arr)
    throughput = len(process_arr) / last_process_finish_time * 100.
    
    print(scheduler.name)
    for process in process_arr:
        print(f"{process.id:04d}:\t{process.at}\t{process.tc}\t{process.cb}\t{process.io}\t{process.static_priority} | {process.finish_time}\t{process.turnaround_time}\t{process.io_time}\t{process.cw}")
        
    print(f"SUM: {last_process_finish_time} {cpu_util:.2f} {io_util:.2f} {avg_tat:.2f} {avg_cw:.2f} {throughput:.3f}")

class Scheduler:
    def __init__(self, name):
        self.name = name
        self.runQ = []
    
    def schedule(self):
        # get the next process from the runQ
        pid = self.runQ.pop(0)
        
    
class DES:
    def __init__(self, process_arr):
        self.process_arr = process_arr
        self.event_queue = []
        for process in process_arr:
            event = (process.at, process.id, 0, process_transition.CREATED_TO_READY)
            heapq.heappush(self.event_queue, event)
        
    def next_event(self):
        if len(self.event_queue) == 0:
            return None

        return heapq.heappop(self.event_queue)
    
    
class RandGenerator:
    def __init__(self, filename):
        self.random_numbers = get_random_numbers(filename)
        self.rand_index = 0
    
    def next(self, burst):
        rand = 1 + (self.random_numbers[self.rand_index] % burst)
        # increment the index
        self.rand_index = (self.rand_index + 1) % len(self.random_numbers)
        return rand

def simulation(des, rand_generator, process_arr, scheduler):
    global last_process_finish_time, cpu_time, io_time
    while (event := des.next_event()):
        clock, pid, time_in_state, transition = event
        remaining_time = process_arr[pid].remaining_time
        
        match transition:
            case process_transition.CREATED_TO_READY:
                print(f"{clock} {pid} {time_in_state}: {transition.name}")
                next_event_for_this_pid = (clock, pid, 0, process_transition.READY_TO_RUNNING)
                process_arr[pid].cw += time_in_state
                heapq.heappush(des.event_queue, next_event_for_this_pid)
            case process_transition.READY_TO_RUNNING:
                cpuburst = rand_generator.next(process_arr[pid].cb)
                cpuburst = min(cpuburst, process_arr[pid].remaining_time)
                print(f"{clock} {pid} {time_in_state}: {transition.name} cb={cpuburst} rem={remaining_time}")
                
                ## if the cpuburst is equal to the remaining time, then the process is done
                if (cpuburst == process_arr[pid].remaining_time):
                    next_event_for_this_pid = (clock + cpuburst, pid, cpuburst, process_transition.DONE)
                else:
                    next_event_for_this_pid = (clock + cpuburst, pid, cpuburst, process_transition.RUNNING_TO_BLOCKED)
                process_arr[pid].remaining_time -= cpuburst
                
                heapq.heappush(des.event_queue, next_event_for_this_pid)
            case process_transition.RUNNING_TO_BLOCKED:
                # accumulate the cpu time
                cpu_time += time_in_state
                ## choose a random number between 1 and io or if io is greater than time left, choose time left
                ioburst = rand_generator.next(process_arr[pid].io)
                
                print(f"{clock} {pid} {time_in_state}: {transition.name} ib={ioburst} rem={remaining_time}")
                next_event_for_this_pid = (clock + ioburst, pid, ioburst, process_transition.BLOCKED_TO_READY)
                heapq.heappush(des.event_queue, next_event_for_this_pid)
            case process_transition.BLOCKED_TO_READY:
                print(f"{clock} {pid} {time_in_state}: {transition.name}")
                
                # accumulate the io time
                process_arr[pid].io_time += time_in_state
                io_time += time_in_state
                
                next_event_for_this_pid = (clock, pid, 0, process_transition.READY_TO_RUNNING)
                heapq.heappush(des.event_queue, next_event_for_this_pid)
            case process_transition.RUNNING_TO_READY:
                pass
            case process_transition.DONE:
                print(f"{clock} {pid} {time_in_state}: {transition.name}")
                process_arr[pid].finish_time = clock
                process_arr[pid].turnaround_time = clock - process_arr[pid].at
                last_process_finish_time = max(last_process_finish_time, clock)
                print(f"DONE {last_process_finish_time} {clock}")


In [9]:
def main():
    rand_generator = RandGenerator("../lab2_assign/rfile")
    process_arr = get_process_array("../lab2_assign/input1")

    des = DES(process_arr)
    scheduler = Scheduler("FCFS")

    simulation(des, rand_generator, process_arr, scheduler)
    print_summary(scheduler, process_arr)
        

In [10]:
main()

1 0 0: CREATED_TO_READY
1 0 0: READY_TO_RUNNING cb=2 rem=200
3 0 2: RUNNING_TO_BLOCKED io=20 rem=198
23 0 20: BLOCKED_TO_READY
23 0 0: READY_TO_RUNNING cb=8 rem=198
31 0 8: RUNNING_TO_BLOCKED io=12 rem=190
43 0 12: BLOCKED_TO_READY
43 0 0: READY_TO_RUNNING cb=10 rem=190
53 0 10: RUNNING_TO_BLOCKED io=17 rem=180
70 0 17: BLOCKED_TO_READY
70 0 0: READY_TO_RUNNING cb=7 rem=180
77 0 7: RUNNING_TO_BLOCKED io=19 rem=173
96 0 19: BLOCKED_TO_READY
96 0 0: READY_TO_RUNNING cb=7 rem=173
103 0 7: RUNNING_TO_BLOCKED io=1 rem=166
104 0 1: BLOCKED_TO_READY
104 0 0: READY_TO_RUNNING cb=9 rem=166
113 0 9: RUNNING_TO_BLOCKED io=19 rem=157
132 0 19: BLOCKED_TO_READY
132 0 0: READY_TO_RUNNING cb=1 rem=157
133 0 1: RUNNING_TO_BLOCKED io=4 rem=156
137 0 4: BLOCKED_TO_READY
137 0 0: READY_TO_RUNNING cb=6 rem=156
143 0 6: RUNNING_TO_BLOCKED io=2 rem=150
145 0 2: BLOCKED_TO_READY
145 0 0: READY_TO_RUNNING cb=2 rem=150
147 0 2: RUNNING_TO_BLOCKED io=14 rem=148
161 0 14: BLOCKED_TO_READY
161 0 0: READY_TO_RUNNI